**Notes : **

1 - custom prediction python script should be provided (scoring.py)

2 - will check if artefacts can be installed to notebook

In [180]:
# import required libraries
import os
import datetime
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    CodeConfiguration,
    ProbeSettings,
    Environment,
    Model,
)
from azure.ai.ml.constants import AssetTypes

from azureml.core import (
    Experiment,
    Workspace,
    Run
)

from azure.identity import DefaultAzureCredential

In [181]:
name = "nnunet-devops-test"
latest_run_id = "d6c83f1e-97a7-48a1-b4db-a308de7bd9e2" # gets the latest if empty

In [182]:
workspace = Workspace.from_config()
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential)

Found the config file in: /config.json


In [183]:
# get experiment
experiment = Experiment(workspace, name)
# get experiment's latest run id
if latest_run_id:
    latest_run = Run(experiment, latest_run_id) 
else:
    latest_run = next(experiment.get_runs())
print(type(latest_run))

# type of latest_run same in both cases

<class 'azureml.core.run.Run'>


In [184]:
model_path = f"azureml://subscriptions/{workspace.subscription_id}/resourcegroups/{workspace.resource_group}/workspaces/{workspace.name}/datastores/{workspace.get_default_datastore().name}/paths/dataset/{latest_run.id}/nnunet_outputs/"

In [186]:
# model registration procedure
model = Model(path=model_path,
                name="iamnewtestmodel",
                description="description-here",
                type=AssetTypes.CUSTOM_MODEL)
registered_model = ml_client.models.create_or_update(model)

In [71]:
# Creating a unique endpoint name with current datetime to avoid conflicts
online_endpoint_name = "nnunet-model-" + datetime.datetime.now().strftime(
    "%m%d%H%M%f"
)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="sample online endpoint for deploying model",
    auth_mode="key",
    tags={"foo": "bar"},
)
print(online_endpoint_name)

nnunet-model-04131722901364


In [72]:
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://nnunet-model-04131722901364.eastus2.inference.ml.azure.com/score', 'openapi_uri': 'https://nnunet-model-04131722901364.eastus2.inference.ml.azure.com/swagger.json', 'name': 'nnunet-model-04131722901364', 'description': 'sample online endpoint for deploying model', 'tags': {'foo': 'bar'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/e5fb5c5f-cb5c-42c8-86b6-cf4fd5aac11b/resourcegroups/azureml-test/providers/microsoft.machinelearningservices/workspaces/nnunet/onlineendpoints/nnunet-model-04131722901364', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/e5fb5c5f-cb5c-42c8-86b6-cf4fd5aac11b/providers/Microsoft.MachineLearningServices/locations/eastus2/mfeOperationsStatus/oe:438978f1-ead1-47c3-ad8e-c77be361c727:baa0e92e-e9c4-4977-8d51-b0d71e27a039?api-version=2022-02-01-preview'}, 'id': '/subscriptions/e5fb5c5f-cb5c-42c8-86b6-cf4fd5aac11b/re

In [73]:
#docker_env.docker.base_image = "nnunet.azurecr.io/training_env" # example : "fastdotai/fastai2:latest"
#docker_env.python.user_managed_dependencies = True

deployment = ManagedOnlineDeployment(
    name=name,
    environment=Environment(image="nnunet.azurecr.io/prediction_env"),
    endpoint_name=online_endpoint_name,
    model=registered_model.id,
    instance_type="Standard_DS1_v2", #"Standard_DS2_v2",
    code_configuration=CodeConfiguration(code="", scoring_script="src/score.py"),
    instance_count=1,
    liveness_probe=ProbeSettings(
        failure_threshold=30,
        success_threshold=1,
        timeout=2,
        period=10,
        initial_delay=2000,
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=10,
        success_threshold=1,
        timeout=10,
        period=10,
        initial_delay=2000,
    ),
)

In [54]:
print("1")

1


In [74]:
ml_client.online_deployments.begin_create_or_update(deployment).result()


Check: endpoint nnunet-model-04131722901364 exists
Your file exceeds 100 MB. If you experience low upload speeds or latency, we recommend using the AzCopy tool for this file transfer. See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.
Uploading avaj1870 (239.95 MBs): 100%|██████████| 239953828/239953828 [00:10<00:00, 22944085.93it/s]


data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


.

In [ ]:
# update deployment to take 100% traffic
endpoint.traffic = {name: 0}
ml_client.begin_create_or_update(endpoint).result()